In [1]:
import numpy as num
import pandas as pd
import os
import nltk as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#Libraries that check the accuracy of the model over test set
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

Code to Determine the Proper directory to your data

In [2]:
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./Leet1.py
./Model.h5
./Model.ipynb
./Fake\Fake.csv
./True\True.csv


Opening Data using Panda

In [3]:
Fake_data = pd.read_csv('./Fake/Fake.csv')
True_data = pd.read_csv('./True/True.csv')

Data Clean-up....Begining

In [5]:
Fake_data.head(2)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"


In [6]:
Fake_data =Fake_data.drop(columns=["date","subject","title"])

In [7]:
True_data = True_data.drop(columns=["date","subject","title"])

Assigning Tags for easy differetiation of News types (either Fake or True) True news = 0 and Fake news = 1
(i.e. Any value approaching 1 indicates Fake news and any Value approaching 0 indicates True news)

In [8]:
True_data['fake'] = 0
Fake_data['fake'] = 1

In [9]:
Fake_data.head(2)

,text,fake
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1


In [10]:
True_data.head()

,text,fake
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0


In [11]:
Fake_data.shape

(23481, 2)

In [12]:
True_data.shape

(21417, 2)

In [13]:
word = "My name is David Adlai Nettey. I do Computer Science"

In [14]:
token = np.word_tokenize(word)

In [15]:
print(token)

['My', 'name', 'is', 'David', 'Adlai', 'Nettey', '.', 'I', 'do', 'Computer', 'Science']


In [16]:
trial = Fake_data['text'][1]

In [17]:
#trial = re.sub('\[[^]]*\]', ' ', trial) # remove punctuations
#trial = re.sub('[^a-zA-Z]',' ',trial)  # replaces non-alphabets with spaces
trial = trial.lower() # Converting from uppercase to lowercase


In [18]:
print(trial)

house intelligence committee chairman devin nunes is going to have a bad day. he s been under the assumption, like many of us, that the christopher steele-dossier was what prompted the russia investigation so he s been lashing out at the department of justice and the fbi in order to protect trump. as it happens, the dossier is not what started the investigation, according to documents obtained by the new york times.former trump campaign adviser george papadopoulos was drunk in a wine bar when he revealed knowledge of russian opposition research on hillary clinton.on top of that, papadopoulos wasn t just a covfefe boy for trump, as his administration has alleged. he had a much larger role, but none so damning as being a drunken fool in a wine bar. coffee boys  don t help to arrange a new york meeting between trump and president abdel fattah el-sisi of egypt two months before the election. it was known before that the former aide set up meetings with world leaders for trump, but team tru

Function necessary for cleaning our Data

In [19]:
def remove_punctuation(text):  #Removing any kind of punctuation present in the data
    return re.sub(r'[^\w\s]','', str(text))

def remove_urls(text):  #Removing urls from the data
    return re.sub(r"http\S+", " ", str(text))
                  
def remove_stopwords(text):  #Removing stopwords(eg. this, that, am, be etc)
    stop = stopwords.words("english")
    final_text = []
    for i in str(text).split():
        if i.strip() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
                  
def tokenize(text):  
    tokens = re.split('\W+',text) #W+ means that either a word character (A-Z) or a dash(-) can go there.
    return tokens

def tokenize_words(text):  #Converting all the text to lower case
    return word_tokenize(text.lower())

                                  
def stemming(text):  #Converting the words into their stem form
    porter_stemmer = PorterStemmer()
    return porter_stemmer.stem(str(text))
                  
def lemmatization(text):  #Applying Lemaatization i.e., converting words into their lemma
    wordnet_lemmatizer = WordNetLemmatizer()
    return wordnet_lemmatizer.lemmatize(str(text))

In [20]:
def clean_text(text):
    #text = remove_punctuation(text)
    #text = remove_urls(text)
    text = remove_stopwords(text)
    text = tokenize_words(text)
    text = stemming(text)
    return lemmatization(text)

In [21]:
True_data['text'] = True_data['text'].apply(lambda x: clean_text(x))

In [22]:
Fake_data['text'] = Fake_data['text'].apply(lambda x: clean_text(x))

In [23]:
Fake_data['text'][0]

"['donald', 'trump', 'wish', 'americans', 'happy', 'new', 'year', 'leave', 'that', '.', 'instead', ',', 'give', 'shout', 'enemies', ',', 'haters', 'dishonest', 'fake', 'news', 'media', '.', 'the', 'former', 'reality', 'show', 'star', 'one', 'job', 'it', '.', 'as', 'country', 'rapidly', 'grows', 'stronger', 'smarter', ',', 'i', 'want', 'wish', 'friends', ',', 'supporters', ',', 'enemies', ',', 'haters', ',', 'even', 'dishonest', 'fake', 'news', 'media', ',', 'happy', 'healthy', 'new', 'year', ',', 'president', 'angry', 'pants', 'tweeted', '.', '2018', 'great', 'year', 'america', '!', 'as', 'country', 'rapidly', 'grows', 'stronger', 'smarter', ',', 'i', 'want', 'wish', 'friends', ',', 'supporters', ',', 'enemies', ',', 'haters', ',', 'even', 'dishonest', 'fake', 'news', 'media', ',', 'happy', 'healthy', 'new', 'year', '.', '2018', 'great', 'year', 'america', '!', 'donald', 'j.', 'trump', '(', '@', 'realdonaldtrump', ')', 'december', '31', ',', '2017trump', 'tweet', 'went', 'welll', 'expe

Combining both fake and true news to form one Dataset

In [24]:
News_data = pd.concat([True_data,Fake_data])

In [ ]:
News_data.shape

: 

In [26]:
News_data.tail(2)

,text,fake
23479,"['21st', 'century', 'wire', 'says', 'al', 'jaz...",1
23480,"['21st', 'century', 'wire', 'says', 'as', '21w...",1


Shuffling our dataset

In [27]:
News_data = shuffle(News_data,random_state=5)

In [28]:
News_data[:9]

,text,fake
10128,"['washington', '(', 'reuters', ')', '-', 'the'...",0
21748,"['it', 'accident', '.', 'it', 'good', 'king', ...",1
14627,"['(', 'this', 'november', '15', 'story', 'corr...",0
17654,"['berlin', '(', 'reuters', ')', '-', 'germany'...",0
6995,"['ted', 'cruz', 'dangerous', 'extremist', 'reg...",1
2653,"['in', 'recent', 'interview', 'fox', 'news', '...",1
2130,"['washington', '(', 'reuters', ')', '-', 'u.s....",0
15083,"['paris', '(', 'reuters', ')', '-', 'eight', '...",0
17176,"['if', 'president', ',', 'arms', 'bear', 'u.s....",1


In [29]:
News_data.tail(2)

,text,fake
18638,"['riyadh', '(', 'reuters', ')', '-', 'saudi', ...",0
14266,"['president', 'obama', ',', 'candidate', 'urge...",1


Data Clean-up...Completed

Building RNN Model

In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

RNN Model

In [31]:
x_train,x_test,y_train,y_test =  train_test_split(News_data['text'],News_data['fake'],test_size=0.3,random_state=8)

In [32]:
x_train.shape

(31428,)

In [33]:
#Tokenize the words into vectors because we can only give numerical data as input to the model

max_vocab = 25000
tokenizer = Tokenizer(num_words = max_vocab)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

#Padding is applied so that we get the same length of input for each article

x_train = pad_sequences(x_train, padding = "post", maxlen = 256)
x_test = pad_sequences(x_test, padding = "post", maxlen = 256)

In [35]:
x_train.shape

(31428, 256)

In [ ]:
"""embedding_matrix = num.zeros((max_vocab, 256))
embedding_layer = tf.keras.layers.Embedding(max_vocab, 256, weights=[embedding_matrix], input_length=25)"""

In [ ]:
"""model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(tf.keras.layers.Dropout(0.45))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.summary()"""

In [36]:
embedding_vector_features = 50
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_vocab, embedding_vector_features, input_length=256))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.LSTM(200, return_sequences=True))
model.add(tf.keras.layers.LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 256, 50)           1250000   
                                                                 
 dropout (Dropout)           (None, 256, 50)           0         
                                                                 
 lstm (LSTM)                 (None, 256, 200)          200800    
                                                                 
 lstm_1 (LSTM)               (None, 100)               120400    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,571,301
Trainable params: 1,571,301
Non-

In [39]:
model.fit(x_train, y_train, epochs = 6, validation_split = 0.10, batch_size = 40, shuffle = True, verbose=1)

Epoch 1/6
708/708 [==============================] - 1217s 2s/step - loss: 0.5295 - accuracy: 0.7084 - val_loss: 0.3751 - val_accuracy: 0.7636
Epoch 2/6
708/708 [==============================] - 25783s 36s/step - loss: 0.2102 - accuracy: 0.9240 - val_loss: 0.1097 - val_accuracy: 0.9675
Epoch 3/6
708/708 [==============================] - 1315s 2s/step - loss: 0.1826 - accuracy: 0.9378 - val_loss: 0.2181 - val_accuracy: 0.9201
Epoch 4/6
708/708 [==============================] - 1316s 2s/step - loss: 0.2143 - accuracy: 0.9122 - val_loss: 0.3257 - val_accuracy: 0.8645
Epoch 5/6
708/708 [==============================] - 1185s 2s/step - loss: 0.1782 - accuracy: 0.9446 - val_loss: 0.1363 - val_accuracy: 0.9660
Epoch 6/6
708/708 [==============================] - 1018s 1s/step - loss: 0.0964 - accuracy: 0.9718 - val_loss: 0.0676 - val_accuracy: 0.9815


In [41]:
model.evaluate(x_test,y_test)

421/421 [==============================] - 104s 246ms/step - loss: 0.0719 - accuracy: 0.9795


[0.07185757905244827, 0.9795100092887878]

Model Accuracy after testing on unseen data is 97.95%

In [40]:
predict = model.predict(x_test)

With our Model Prediction, 1 == Fake news and 0 == Factual news. eg: if the model predicts 80% it definately means Fake news and if the model predicts 20% its definately Factual news.

In [47]:
predict[:2]

array([[0.97274655],
       [0.00613561]], dtype=float32)

In [46]:
y_train[0:2]

20739    1
20005    0
Name: fake, dtype: int64

In [67]:
def news_preperation(news:str):
    news = clean_text(news)
    tokenizer.fit_on_texts(news)
    news = tokenizer.texts_to_sequences(news)
    news = pad_sequences(news, padding= "post", maxlen = 256)
    results = model.predict(news)
    result = results[0]
    return "{:.2f}".format(result[0]*100)

In [78]:
res = news_preperation("There is a War in Ukraine")
res = float(res)
false = res
true = 100.00 - res
print("Fake news Probability: "+ str(false) + " True News Probability: "+ str(true))

Fake news Probability: 98.93 True News Probability: 1.0699999999999932


Saving the AI Model.

In [57]:
model.save("Model.h5")

In [58]:
Fmodel = tf.keras.models.load_model("Model.h5")

In [60]:
Fmodel.evaluate(x_test,y_test)

421/421 [==============================] - 100s 235ms/step - loss: 0.0719 - accuracy: 0.9795


[0.07185757905244827, 0.9795100092887878]